In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

# Total Small Business with Lat & Long

In [1]:
totalrecord = pd.read_csv('total_business.csv')

In [2]:
business = totalrecord.drop_duplicates(subset='placekey', keep='first')

In [3]:
closebusiness = pd.read_csv('closed_business.csv')

In [4]:
Totaldata = business[['placekey','location_name','top_category','sub_category','latitude','longitude','postal_code']]
Closedata = closebusiness[['placekey','location_name','top_category','sub_category','latitude','longitude','postal_code']]

In [75]:
Closedata['status'] = 'close'
Closedata.head()

,placekey,location_name,top_category,sub_category,latitude,longitude,postal_code,status
0,235-222@627-s8j-3kf,Zabb City,Restaurants and Other Eating Places,Full-Service Restaurants,40.731610,-73.986237,10003,close
1,22g-223@627-s8h-zs5,La Paloma,Restaurants and Other Eating Places,Full-Service Restaurants,40.734824,-73.985694,10003,close
2,zzw-222@627-s8k-qpv,New Saigon,Restaurants and Other Eating Places,Full-Service Restaurants,40.725014,-73.987395,10003,close
3,225-223@627-s8j-4vz,Original Tony's House of Pizza,Restaurants and Other Eating Places,Full-Service Restaurants,40.735386,-73.979855,10010,close
4,233-222@627-s8k-r49,Calcutta,Restaurants and Other Eating Places,Full-Service Restaurants,40.726829,-73.987380,10003,close


In [76]:
Totaldata = pd.merge(Totaldata,Closedata[['placekey','status']],on='placekey',how='left')

In [98]:
#Totaldata.to_csv('Total_status.csv')
Totaldata = Totaldata[Totaldata['latitude']>0]
len(Totaldata)

2186

In [99]:
Totaldata_points = gpd.GeoDataFrame(Totaldata,geometry=gpd.points_from_xy(Totaldata['longitude'],Totaldata['latitude']),crs='EPSG:4326')

In [100]:
Totaldata_points .to_file('total_data.shp',
           driver='ESRI Shapefile',
           encoding='utf-8')

# Variable 1: Proximity to major transport stations (Accessibility)

In [101]:
# Metro Station & Bus Stop
Busstop = gpd.read_file('selected_busstop.shp')
Subway = gpd.read_file('selected_subway.shp')

In [106]:
print(len(Busstop))
print(len(Subway))

431
276


In [8]:
# Define a distance function of two geopoints
from math import radians, cos, sin, asin, sqrt
def geodistance(lng1,lat1,lng2,lat2):  
    lng1,lat1,lng2,lat2 = map(radians,[float(lng1), float(lat1), float(lng2), float(lat2)])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    distance=2*asin(sqrt(a))*6371*1000
    distance=round(distance,2)
    return distance

In [ ]:
lst=[]
n=0
for i in range(len(Totaldata_points)):
    dist_lst=[]
    lng1=Totaldata_points.iloc[i]['geometry'].x
    lat1=Totaldata_points.iloc[i]['geometry'].y
    for j in range(len(Busstop)):
        lng2=Busstop.iloc[j]['geometry'].x
        lat2=Busstop.iloc[j]['geometry'].y
        distance = geodistance(lng1,lat1,lng2,lat2)
        dist_lst.append(distance)
        n+=1
        if n%10000==0:
            print(n)
    shortest_dist = min(dist_lst)
    lst.append(shortest_dist)
    
Totaldata_points['distance_to_busstop'] = pd.Series(lst, index=Totaldata_points.index)

In [125]:
#Totaldata_points.to_csv('Totaldata_points.csv')
#Totaldata_points = pd.read_csv('Totaldata_points.csv')

# Variable 2: Nearby Counts

In [10]:
Totaldata_points['sub_category'].unique()

array(['Snack and Nonalcoholic Beverage Bars', 'Full-Service Restaurants',
       'Supermarkets and Other Grocery (except Convenience) Stores',
       'Limited-Service Restaurants', 'Convenience Stores'], dtype=object)

In [4]:
zone1 = Totaldata_points[Totaldata_points['postal_code'].isin([10003,10010])]
zone2 = Totaldata_points[Totaldata_points['postal_code'].isin([11355,11365])]
zone3 = Totaldata_points[Totaldata_points['postal_code'].isin([11237,11206])]
zone4 = Totaldata_points[Totaldata_points['postal_code'].isin([10467,10462])]
print(len(zone1))
print(len(zone2))
print(len(zone3))
print(len(zone4))

1090
274
513
309


In [23]:
n=0
same_lst=[]
different_lst=[]

for i in range(len(zone4)):
    lng1=zone4.iloc[i]['longitude']
    lat1=zone4.iloc[i]['latitude']
    same_500=-1
    different_500=0
    for j in range(len(zone4)):
        lng2=zone4.iloc[j]['longitude']
        lat2=zone4.iloc[j]['latitude']
        distance = geodistance(lng1,lat1,lng2,lat2)
        if zone4.iloc[i]['sub_category']==zone4.iloc[j]['sub_category'] and distance<=500:
            same_500+=1
        elif zone4.iloc[i]['sub_category']!=zone4.iloc[j]['sub_category'] and distance<=500:
            different_500+=1
        else:
            pass
    same_lst.append(same_500)
    different_lst.append(different_500)
    n+=1
    print(n)
zone4['500_same'] = pd.Series(same_lst,index=zone4.index)
zone4['500_different'] = pd.Series(different_lst,index=zone4.index)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [25]:
zone = pd.concat([zone1,zone2,zone3,zone4])

In [41]:
#zone.to_csv('Totaldata_points3.csv',index=False)
Totaldata_points2 = pd.read_csv('Totaldata_points2.csv')
Totaldata_points2 = Totaldata_points2[['placekey','mix_index','commercial_percent','building_d']]
Totaldata_points3 = pd.merge(zone,Totaldata_points2,on='placekey') 
Totaldata_points3.head()

,placekey,location_name,top_category,sub_category,latitude,longitude,postal_code,status,geometry,distance_to_subway,distance_to_busstop,500_same,500_different,mix_index,commercial_percent,building_d
0,zzy-223@627-wc7-st9,The Alley LuJiaoXiang,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,40.729621,-73.991264,10003.0,NaN,POINT (-73.991264 40.729621),40.50,319.47,56,335,2.092311,0.652666,0.517760
1,zzw-225@627-s8k-qs5,Mayahuel Mariposa,Restaurants and Other Eating Places,Full-Service Restaurants,40.727036,-73.988199,10003.0,NaN,POINT (-73.98819899999999 40.727036),405.36,59.28,281,122,1.958600,0.545428,0.479001
2,22d-224@627-s8h-whq,Godiva Cafe,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,40.740358,-73.989665,10010.0,NaN,POINT (-73.989665 40.740358),44.56,107.40,42,251,1.914169,0.660980,0.426000
3,zzw-225@627-s8j-49z,Gramercy Food Market,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,40.736846,-73.981304,10010.0,NaN,POINT (-73.98130399999999 40.736846),530.88,88.15,5,169,2.055004,0.324798,0.427163
4,zzy-223@627-s8j-dqf,The Halal Kitchen,Restaurants and Other Eating Places,Limited-Service Restaurants,40.735139,-73.992067,10003.0,NaN,POINT (-73.99206700000001 40.735139),76.70,105.80,35,202,2.001040,0.529304,0.540477


In [46]:
aa = gpd.read_file('total_data.shp')
aa = aa[['placekey','is_corner']].fillna(0)
Totaldata_points4 = pd.merge(Totaldata_points3,aa,on='placekey') 

In [48]:
Totaldata_points4.to_csv('Indicator.csv',index=False)
Totaldata_points4.head()

,placekey,location_name,top_category,sub_category,latitude,longitude,postal_code,status,geometry,distance_to_subway,distance_to_busstop,500_same,500_different,mix_index,commercial_percent,building_d,is_corner
0,zzy-223@627-wc7-st9,The Alley LuJiaoXiang,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,40.729621,-73.991264,10003.0,NaN,POINT (-73.991264 40.729621),40.50,319.47,56,335,2.092311,0.652666,0.517760,0.0
1,zzw-225@627-s8k-qs5,Mayahuel Mariposa,Restaurants and Other Eating Places,Full-Service Restaurants,40.727036,-73.988199,10003.0,NaN,POINT (-73.98819899999999 40.727036),405.36,59.28,281,122,1.958600,0.545428,0.479001,0.0
2,22d-224@627-s8h-whq,Godiva Cafe,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,40.740358,-73.989665,10010.0,NaN,POINT (-73.989665 40.740358),44.56,107.40,42,251,1.914169,0.660980,0.426000,0.0
3,zzw-225@627-s8j-49z,Gramercy Food Market,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,40.736846,-73.981304,10010.0,NaN,POINT (-73.98130399999999 40.736846),530.88,88.15,5,169,2.055004,0.324798,0.427163,0.0
4,zzy-223@627-s8j-dqf,The Halal Kitchen,Restaurants and Other Eating Places,Limited-Service Restaurants,40.735139,-73.992067,10003.0,NaN,POINT (-73.99206700000001 40.735139),76.70,105.80,35,202,2.001040,0.529304,0.540477,0.0


# Variable 3: Street width and Proximity to healthcare facilities

### 1) add street width

In [7]:
street_width = gpd.read_file('st_width.shp')
street_width = street_width[['st_width','placekey']]
street_width

,st_width,placekey
0,40.0,zzy-223@627-wc7-st9
1,30.0,zzw-225@627-s8k-qs5
2,44.0,22d-224@627-s8h-whq
3,60.0,227-223@627-s9g-2rk
4,58.0,zzw-225@627-s8j-49z
...,...,...
2181,30.0,225-222@627-rw6-qfz
2182,34.0,22j-222@627-rwc-py9
2183,40.0,223-222@627-rwy-6ff
2184,42.0,zzw-223@627-rwb-5mk


In [46]:
Totaldata_points5 = pd.merge(indiactor_old,street_width,on='placekey') 

### 2) add proximity to healthcare facilities

In [30]:
health = pd.read_csv('hospital&drug.csv')

In [31]:
health['top_category'].unique()

array(['Health and Personal Care Stores',
       'General Medical and Surgical Hospitals'], dtype=object)

In [55]:
health['sub_category'].unique()

array(['Pharmacies and Drug Stores',
       'General Medical and Surgical Hospitals'], dtype=object)

In [33]:
hospital = health[health['top_category']=='General Medical and Surgical Hospitals']
len(hospital)

1043

In [34]:
drugstore = health[health['top_category']=='Health and Personal Care Stores']
len(drugstore)

5859

In [36]:
hospital.to_csv('hospital.csv')
drugstore.to_csv('drugstore.csv')

In [37]:
hospital = gpd.read_file('selected_hospital(1000).shp')
pharmacy = gpd.read_file('selected_pharmacy.shp')

In [38]:
print(len(hospital))
print(len(pharmacy))

83
449


In [39]:
from math import radians, cos, sin, asin, sqrt
def geodistance(lng1,lat1,lng2,lat2):  
    lng1,lat1,lng2,lat2 = map(radians,[float(lng1), float(lat1), float(lng2), float(lat2)])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    distance=2*asin(sqrt(a))*6371*1000
    distance=round(distance,2)
    return distance

In [48]:
Totaldata_points = gpd.GeoDataFrame(Totaldata_points5,geometry=gpd.points_from_xy(Totaldata_points5['longitude'],Totaldata_points5['latitude']),crs='EPSG:4326')

In [49]:
Totaldata_points .to_file('total_data2.shp',
           driver='ESRI Shapefile',
           encoding='utf-8')

In [52]:
lst=[]
n=0
for i in range(len(Totaldata_points)):
    dist_lst=[]
    lng1=Totaldata_points.iloc[i]['geometry'].x
    lat1=Totaldata_points.iloc[i]['geometry'].y
    for j in range(len(pharmacy)):
        lng2=pharmacy.iloc[j]['geometry'].x
        lat2=pharmacy.iloc[j]['geometry'].y
        distance = geodistance(lng1,lat1,lng2,lat2)
        dist_lst.append(distance)
        n+=1
        if n%10000==0:
            print(n)
    shortest_dist = min(dist_lst)
    lst.append(shortest_dist)
    
Totaldata_points['distance_to_pharmacy'] = pd.Series(lst, index=Totaldata_points.index)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000


In [54]:
Totaldata_points.to_csv('spatial_variables.csv')